In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import torch
from inpainting.inpainters.rgb_misgan import RGBMisganInpainterInterface
from inpainting.inpainters.mnist import MNISTMisganInpainterInterface
from inpainting.datasets.celeba import train_val_datasets as celeba_train_val_datasets
from inpainting.datasets.mnist import train_val_datasets as mnist_train_val_datasets
from pathlib import Path
from inpainting.datasets.utils import RandomRectangleMaskConfig
from inpainting.datasets import mask_coding as mc
from torchvision.datasets import MNIST, FashionMNIST
from torch.utils.data import DataLoader, TensorDataset
import pickle
from tqdm import tqdm
import numpy as np
from inpainting.visualizations import visualizations_utils as vis
from inpainting.visualizations.digits import digit_with_mask as vis_digit_mask
import matplotlib.pyplot as plt
from inpainting.utils import predictions_for_entire_loader
from scripts import common
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cpu")

In [ ]:
ds_train, ds_val = mnist_train_val_datasets(
    Path.home() / "uj/data/",
    mask_configs=[

        RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 14, 14, 0,0),
    ],
    ds_type=MNIST
)

In [ ]:
batch_size=16
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_val = DataLoader(ds_val, batch_size, shuffle=False)

In [ ]:
mfa = common.mfa_from_path("/home/mprzewiezlikowski/uj/gmm_missing/models/mnist_28_28")

In [ ]:
for (X, J), Y in dl_train:
#     X, J = X[:1], J[:1]
    print("predicting")
    P, M, A, D = mfa(X, J)
    Pb, Mb, Ab, Db = mfa.fb(X, J)
    print("predicted")
    break
    
N = 5
print(vis.row_length(*[t[0] for t in [X, J, P, M, A, D, Y]]))

fig, ax = plt.subplots(N, vis.row_length(*[t[0] for t in [X, J, P, M, A, D, Y]]), figsize=(15, 10))
fig.suptitle("separate")
for i, (x,j,p,m,a,d,y) in enumerate(list(zip(X,J,P,M,A,D,Y))[:N]):
    x,j,p,m,a,d = [t.numpy() for t in [x,j,p,m,a,d]]
    vis.visualize_sample(
        x,j,p,m,a,d,y,
        drawing_fn=vis_digit_mask,
        title_prefixes=dict(),
        ax_row=ax[i]
    )
    
plt.show()

print([t.shape for t in [P, M, A, D]])
print([t.shape for t in [Pb, Mb, Ab, Db]])
fig, ax = plt.subplots(N, vis.row_length(*[t[0] for t in [X, J, Pb, Mb, Ab, Db, Y]]), figsize=(15, 10))
fig.suptitle("batch")
for i, (x,j,p,m,a,d,y) in enumerate(list(zip(X,J,Pb,Mb,Ab,Db,Y))[:N]):
    x,j,p,m,a,d = [t.numpy() for t in [x,j,p,m,a,d]]
    vis.visualize_sample(
        x,j,p,m,a,d,y,
        drawing_fn=vis_digit_mask,
        title_prefixes=dict(),
        ax_row=ax[i]
    )
    
plt.show()
    
# vis.visa